In [1]:
# TESTES REALIZADOS ABAIXO NÃO UTILIZARAM TIME SERIES SPLIT DEVIDO A ERRO EM TENTATIVAS ANTERIORES, AO INVES DISSO FORAM UTILIZADAS DIVISÃO 80/20 RESPEITANDO A ORDEM TEMPORAL

In [2]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [4]:
# Carregar dados
file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

In [5]:
# Processar dados
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

In [6]:
# Divisão dos dados respeitando ordem temporal
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]


In [7]:
# Parâmetros para seleção de features
k_values = [5, 10, 15]
percentile_values = [10, 20, 30]
melhor_r2 = float('-inf')
melhor_X_train = None
melhor_X_test = None
melhores_features = None

In [8]:
# FEATURE SELECTION

for k in k_values:
    for p in percentile_values:
        skb = SelectKBest(score_func=f_regression, k=min(k, X_train.shape[1]))
        sp = SelectPercentile(score_func=f_regression, percentile=p)
        
        X_train_kbest = skb.fit_transform(X_train, y_train)
        X_test_kbest = skb.transform(X_test)
        selected_features_kbest = X_train.columns[skb.get_support()]
        
        X_train_percentile = sp.fit_transform(X_train, y_train)
        X_test_percentile = sp.transform(X_test)
        selected_features_percentile = X_train.columns[sp.get_support()]
        
        for X_train_selected, X_test_selected, selected_features in [
            (X_train_kbest, X_test_kbest, selected_features_kbest), 
            (X_train_percentile, X_test_percentile, selected_features_percentile)]:
            
            model = LinearRegression()
            model.fit(X_train_selected, y_train)
            y_pred = model.predict(X_test_selected)
            r2 = r2_score(y_test, y_pred)
            
            if r2 > melhor_r2:
                melhor_r2 = r2
                melhor_X_train = X_train_selected
                melhor_X_test = X_test_selected
                melhores_features = selected_features

print(f"Melhores features selecionadas: {list(melhores_features)}")

Melhores features selecionadas: ['Valor IPCA', 'IPCA Mês -10', 'IPCA Mês -9', 'IPCA Mês -8', 'IPCA Mês -7', 'IPCA Mês -6', 'IPCA Mês -5', 'IPCA Mês -4', 'IPCA Mês -3', 'IPCA Mês -2', 'SELIC Mês -2', 'SELIC Mês -1', 'SELIC Mês 0', 'IPP NOVO Mês -10', 'DESEMPREGO Mês -11', 'DESEMPREGO Mês -10', 'DESEMPREGO Mês -5', 'DESEMPREGO Mês -4', 'DESEMPREGO Mês -3', 'DESEMPREGO Mês -2', 'DESEMPREGO Mês -1', 'DESEMPREGO Mês 0']


In [9]:
# Modelos a serem testados
modelos = {
    "Regressão Linear": LinearRegression(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

In [10]:
# GridSearch para cada modelo
for nome, modelo in modelos.items():
    parametros = {}
    if isinstance(modelo, SVR):
        parametros = {'C': [0.1, 1, 10], 'epsilon': [0.001, 0.01, 0.1], 'kernel': ['linear', 'rbf']}
    elif isinstance(modelo, DecisionTreeRegressor):
        parametros = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
    elif isinstance(modelo, RandomForestRegressor):
        parametros = {'n_estimators': [100, 200], 'max_depth': [None, 10], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}
    elif isinstance(modelo, GradientBoostingRegressor):
        parametros = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]}
    
    grid_search = GridSearchCV(modelo, parametros, cv=5, scoring= 'r2', refit=True)
    grid_search.fit(melhor_X_train, y_train)
    
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(melhor_X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Modelo: {nome}")
    print(f"Melhores parâmetros: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"R²: {r2:.4f}")
    print("="*50)

Modelo: Regressão Linear
Melhores parâmetros: {}
MSE: 0.0034
R²: 0.9505
Modelo: SVR
Melhores parâmetros: {'C': 10, 'epsilon': 0.01, 'kernel': 'linear'}
MSE: 0.0044
R²: 0.9369
Modelo: Decision Tree
Melhores parâmetros: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
MSE: 0.0120
R²: 0.8261
Modelo: Random Forest
Melhores parâmetros: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
MSE: 0.0072
R²: 0.8962
Modelo: Gradient Boosting
Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
MSE: 0.0120
R²: 0.8261


In [ ]:
#ABAIXO TESTE REALIDADO SEM FEATURE SELECTION E UTILIZANDO A MESMA DIVISÃO DE TREINO E TESTE - MSE PARA MELHOR MODELO

In [11]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Carregar dados
file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Processar dados
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

# Divisão dos dados respeitando ordem temporal
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Treinamento e avaliação dos modelos
models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R2": r2}

# Exibir resultados
for model, scores in results.items():
    print(f"{model}: MSE = {scores['MSE']:.4f}, R2 = {scores['R2']:.4f}")


Linear Regression: MSE = 0.5631, R2 = -7.1503
SVR: MSE = 0.0553, R2 = 0.1996
Decision Tree: MSE = 0.0165, R2 = 0.7605
Random Forest: MSE = 0.0089, R2 = 0.8713
Gradient Boosting: MSE = 0.0122, R2 = 0.8227


In [ ]:
#ABAIXO TESTE REALIDADO SEM FEATURE SELECTION E UTILIZANDO A MESMA DIVISÃO DE TREINO E TESTE - R2 PARA MELHOR MODELO

In [13]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Carregar dados
file_path = r'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Processar dados
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

# Divisão dos dados respeitando ordem temporal
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Definição dos modelos e hiperparâmetros
models_params = {
    "SVR": (SVR(), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}),
    "Decision Tree": (DecisionTreeRegressor(), {"max_depth": [3, 5, 10]}),
    "Random Forest": (RandomForestRegressor(), {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 10]}),
    "Gradient Boosting": (GradientBoostingRegressor(), {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 0.2]})
}

best_models = {}
for name, (model, params) in models_params.items():
    grid_search = GridSearchCV(model, params, scoring='r2', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[name] = {
        "Best Params": grid_search.best_params_,
        "Best R2": grid_search.best_score_
    }

# Exibir melhores modelos e parâmetros
for model, result in best_models.items():
    print(f"{model}: Melhor R2 = {result['Best R2']:.4f}, Melhores Parâmetros = {result['Best Params']}")

SVR: Melhor R2 = -2.5388, Melhores Parâmetros = {'C': 1, 'kernel': 'rbf'}
Decision Tree: Melhor R2 = -1.2714, Melhores Parâmetros = {'max_depth': 3}
Random Forest: Melhor R2 = -0.5060, Melhores Parâmetros = {'max_depth': 5, 'n_estimators': 100}
Gradient Boosting: Melhor R2 = -0.1189, Melhores Parâmetros = {'learning_rate': 0.2, 'n_estimators': 200}
